In [2]:
!pip install geopy

In [3]:
import pandas as pd
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
#import plotly_express as px
#import geopandas as gpd
import geopy as gp
from geopy.geocoders import GoogleV3

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_csv('/content/drive/MyDrive/reduced_data.csv')

In [8]:
df = df[df.Value > 0]

In [9]:
df["temp"] = np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_grouped = df.groupby(['Latitude', 'Longitude', 'Height']).agg({'temp':'size', 'Value':'mean'}).rename(columns={'Value':'Value_mean','temp':'number of measurements'}).reset_index()

In [11]:
df_grouped # dataframe z uśrednionymi wartościami dla każdego miejsca pobrań(Value_mean), number of measurements to liczba wykonanych pomiarów

,Latitude,Longitude,Height,number of measurements,Value_mean
0,-90.000000,37.442836,5.0,2,10.000000
1,-47.392879,127.472214,173.0,20,25.250000
2,-36.852563,174.763318,82.0,1,47.000000
3,-36.852563,174.763340,82.0,1,46.000000
4,-36.852542,174.763392,81.0,1,49.000000
...,...,...,...,...,...
70440,52.395588,4.875916,103.0,14,21.928571
70441,52.395607,4.874018,-32.0,257,25.599222
70442,60.202004,24.921344,44.0,1,54.000000
70443,90.000000,123.456000,123.0,1,123.000000


In [12]:
df_grouped.sort_values(by = ['Latitude', 'Longitude'], inplace = True)

In [13]:
df_grouped

,Latitude,Longitude,Height,number of measurements,Value_mean
0,-90.000000,37.442836,5.0,2,10.000000
1,-47.392879,127.472214,173.0,20,25.250000
2,-36.852563,174.763318,82.0,1,47.000000
3,-36.852563,174.763340,82.0,1,46.000000
4,-36.852542,174.763392,81.0,1,49.000000
...,...,...,...,...,...
70440,52.395588,4.875916,103.0,14,21.928571
70441,52.395607,4.874018,-32.0,257,25.599222
70442,60.202004,24.921344,44.0,1,54.000000
70443,90.000000,123.456000,123.0,1,123.000000


In [31]:
def get_country(row):
    locator = Nominatim(user_agent="ekspllo3", timeout=10)
    pos = str(row['Latitude']) + ', ' + str(row['Longitude'])
    location = locator.reverse(pos, timeout = 10, language="en")
    try:
      return location.raw['address']['country']
    except:
      return "no data" 

In [41]:
import math
import statistics
from tqdm import tqdm
data = []

Latitude, Longitude = 0, 0
countries = []
curr_country = "no data"

#df__dict = df.to_dict('records')
df_grouped_dict = df_grouped.to_dict('records')

for row in tqdm(df_grouped_dict):
    if Latitude == 0 and Longitude == 0:
      Latitude, Longitude = row['Latitude'], row['Longitude']
      curr_country = get_country(row)
      countries.append(curr_country)
    else:
        distance = math.sqrt((Latitude - row['Latitude'])**2 + (Longitude- row['Longitude'])**2) 
        if distance < 0.5:  #wtedy są blisko i można potraktować je z pewnością jako ten sam kraj (czyli przyłączamy wartości średnie do)
            countries.append(curr_country)
        else:

            Latitude, Longitude = row['Latitude'], row['Longitude']
            curr_country = get_country(row)
            countries.append(curr_country)

cdf = df_grouped
cdf['Country'] = countries


100%|██████████| 70445/70445 [00:50<00:00, 1381.55it/s]


In [40]:
print(cdf)

,Latitude,Longitude,Height,number of measurements,Value_mean,Country
0,-90.000000,37.442836,5.0,2,10.000000,no data
1,-47.392879,127.472214,173.0,20,25.250000,no data
2,-36.852563,174.763318,82.0,1,47.000000,New Zealand
3,-36.852563,174.763340,82.0,1,46.000000,New Zealand
4,-36.852542,174.763392,81.0,1,49.000000,New Zealand
...,...,...,...,...,...,...
70440,52.395588,4.875916,103.0,14,21.928571,Netherlands
70441,52.395607,4.874018,-32.0,257,25.599222,Netherlands
70442,60.202004,24.921344,44.0,1,54.000000,Finland
70443,90.000000,123.456000,123.0,1,123.000000,no data


In [45]:
cdf.sort_values(by = ['Country'], inplace = True)

In [46]:
cdf['Country'].value_counts()

United States    61223
Japan             8167
South Korea        452
New Zealand        247
China              159
Netherlands        121
Slovenia            54
no data              9
Canada               4
Germany              4
Italy                2
Finland              1
Somalia              1
Spain                1
Name: Country, dtype: int64

In [49]:
countries = cdf['Country'].unique()

In [50]:
countries

array(['Canada', 'China', 'Finland', 'Germany', 'Italy', 'Japan',
       'Netherlands', 'New Zealand', 'Slovenia', 'Somalia', 'South Korea',
       'Spain', 'United States', 'no data'], dtype=object)

In [44]:
cdf.describe()

,Latitude,Longitude,Height,number of measurements,Value_mean
count,70445.000000,70445.000000,70445.000000,7.044500e+04,70445.000000
mean,35.748421,-82.930392,104.024998,5.082449e+02,57.047007
std,5.202132,86.442267,117.307459,1.847460e+04,249.638167
min,-90.000000,-122.851453,-1809.000000,1.000000e+00,1.000000
25%,34.122169,-122.128059,20.000000,2.000000e+00,24.750000
50%,34.176193,-118.261475,104.000000,4.000000e+00,30.250000
75%,37.738979,-118.093719,162.000000,3.600000e+01,34.050000
max,90.000000,174.765607,6228.000000,1.257088e+06,33475.310345


In [57]:
def get_info(df):
  print(f"Total number of measurements : {df['number of measurements'].sum()}")
  print(f"Min Latitude : {df['Latitude'].min()} | Max Latitude : {df['Latitude'].max()}")
  print(f"Min Longitude : {df['Longitude'].min()} | Max Longitude : {df['Longitude'].max()}")
  print(f"Min Height : {df['Height'].min()} | Max Height : {df['Height'].max()}")
  print(f"Mean value of radiation : {df['Value_mean'].mean()}")
  print(f"Max value of radiation : {df['Value_mean'].max()}")

In [66]:
for country in countries:
  print(country + ":" + '\n')
  get_info(cdf[cdf.Country == country])
  print('\n' + "-" *50 + '\n')

Canada:

Total number of measurements : 4
Min Latitude : 43.3902971572416 | Max Latitude : 43.3920435590073
Min Longitude : -79.8252136094009 | Max Longitude : -79.8239368779098
Min Height : 2.0 | Max Height : 2.0
Mean value of radiation : 124.5
Max value of radiation : 337.0

--------------------------------------------------

China:

Total number of measurements : 31011
Min Latitude : 22.284233 | Max Latitude : 22.312658
Min Longitude : 114.145004 | Max Longitude : 114.184113
Min Height : 2.0 | Max Height : 199.0
Mean value of radiation : 71.66703198524367
Max value of radiation : 89.65

--------------------------------------------------

Finland:

Total number of measurements : 1
Min Latitude : 60.2020037715231 | Max Latitude : 60.2020037715231
Min Longitude : 24.9213437877441 | Max Longitude : 24.9213437877441
Min Height : 44.0 | Max Height : 44.0
Mean value of radiation : 54.0
Max value of radiation : 54.0

--------------------------------------------------

Germany:

Total number